In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D#二维卷积
from tensorflow.keras import Model

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[:, tf.newaxis]
x_train[1].shape
print(x_train.shape)

(60000, 28, 28, 1)


In [6]:
x_train.shape[0]

60000

In [9]:
# 使用 tf.data 来将数据集切分为 batch 以及混淆数据集：

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [12]:
train_ds.shape


AttributeError: 'BatchDataset' object has no attribute 'shape'

In [15]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='selu',input_shape=[1,28,28]),
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='selu'),
    keras.layers.Dense(10,activation='softmax')
])

model.compile(loss ='sparse_categorical_crossentropy',
             optimizer='adam',metrics=['accuracy'])
model.summary()

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_3/MaxPool' (op: 'MaxPool') with input shapes: [?,1,28,32].

In [11]:
epochs = 100
history = model.fit(train_ds,
                  steps_per_epoch = x_train.shape[0]  //32,
                  epochs= epochs,
                  validation_data = test_ds,
                  validation_steps = x_test.shape[0]//32
                             )

ValueError: in converted code:

    /home/zo1zu22/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /home/zo1zu22/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py:2410 _standardize_tensors
        exception_prefix='input')
    /home/zo1zu22/environment/tf2_py3/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:582 standardize_input_data
        str(data_shape))

    ValueError: Error when checking input: expected conv2d_input to have shape (28, 28, 1) but got array with shape (1, 28, 28)


In [4]:
#使用 Keras 模型子类化（model subclassing） API 构建 tf.keras 模型：

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 1, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [5]:
# 为训练选择优化器与损失函数：

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [6]:
#选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果。
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
# 使用 tf.GradientTape 来训练模型：

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [12]:
# 测试模型：
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [13]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [14]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in converted code:

    <ipython-input-12-f83207bac040>:4 test_step  *
        predictions = model(images)
    /Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-4-0f9cc5954f4c>:12 call  *
        x = self.conv1(x)
    /Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:812 __call__
        self.name)
    /Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/input_spec.py:213 assert_input_compatibility
        ' but received input with shape ' + str(shape))

    ValueError: Input 0 of layer conv2d is incompatible with the layer: expected axis -1 of input shape to have value 1 but received input with shape [32, 1, 28, 28]
